In [1]:
import os
import pandas as pd
import numpy as np

##### data directories

In [3]:
currDir = os.getcwd()
rootDir = os.path.abspath(os.path.join(currDir,'..'))

dataDir = os.path.abspath(os.path.join(rootDir,'data'))
rawDataDir = os.path.abspath(os.path.join(dataDir,'raw'))
interimDataDir = os.path.abspath(os.path.join(dataDir,'interim'))
finalDataDir = os.path.abspath(os.path.join(dataDir,'final'))
errorLog = os.path.abspath(os.path.join(dataDir,'error_log'))

#### read player bio data

In [8]:
file = f"{rawDataDir}/all_NBA_ABA_players.csv"
df = pd.read_csv(file)

# keep a list of each player's unique identifier
index = df['index'].values

print("No of distinct players:",df['index'].nunique())

No of distinct players: 4800


##### aggregating scraped player data

In [64]:
#combine all files in the directory
files = os.listdir(rawDataDir)
files.sort()
files = files[2:]

test = pd.concat([pd.read_csv(f"{rawDataDir}/{f}") 
                  for f in files], sort = False)
#export to csv
# combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

print(f"No of game logs: {test.shape[0]:,}")

No of game logs: 1,830,907


##### identifying players whose data has not been scraped

In [38]:
# players in the error log
logger = f"{errorLog}/log.txt"

with open(logger,"r") as logs:
    lines = [[x.strip() for x in line.strip().split(',')] 
             for line in logs]

cols = ['index','name','reason']
error_df = pd.DataFrame(lines,columns=cols)
error_idx = error_df['index'].values

print(error_df.shape)

(103, 3)


In [40]:
error_df.head()

,index,name,reason
0,alexacl01,Cliff Alexander,error in playoff game logs
1,anigbik01,Ike Anigbogu,error in playoff game logs
2,babbch01,Chris Babb,error in playoff game logs
3,bairsca01,Cameron Bairstow,error in playoff game logs
4,bambamo01,Mohamed Bamba,error in playoff game logs
